In [ ]:
import numpy as np 
import pandas as pd 

df = pd.read_csv("./Datasets/train.csv")

In [ ]:
df.head()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader 
from torch.utils.data import Dataset

In [ ]:
from sklearn.model_selection import train_test_split
train,test= train_test_split(df, test_size=0.4)
val,test= train_test_split(test, test_size=0.5)

In [ ]:
class MyDataset(Dataset):
  def __init__(self,df):

    x=df.iloc[:,1:786].values
    y=df.iloc[:,0].values
    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y_train=F.one_hot(torch.tensor(y,dtype=torch.float32).type(torch.LongTensor)).type(torch.FloatTensor)

 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [ ]:
data=MyDataset(train)
train_loader=DataLoader(data,batch_size=10)
data=MyDataset(val)
val_loader=DataLoader(data,batch_size=10)
data=MyDataset(test)
test_loader=DataLoader(data,batch_size=10)

In [ ]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fco = nn.Linear(50, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fco(x)
        x= self.softmax(x)
        return x 

In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 784 
num_classes = 10
learning_rate = 0.0001
num_epochs = 20

In [ ]:
model = NN(input_size=input_size, num_classes=num_classes)#.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model)

In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            _, truth = y.max(1)
            num_correct += (predictions == truth).sum()
            num_samples += predictions.size(0)
    model.train()
    return float(num_correct) / float(num_samples) * 100

In [33]:
for epoch in range(num_epochs):
    train_ac=check_accuracy(train_loader, model)
    val_ac=check_accuracy(val_loader, model)
    print(f"Epoch: {epoch}",f"Train_accuracy: {train_ac}",f"Cross_validation_accuracy: {val_ac}")
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        #data = data.to(device=device)
        #targets = targets.to(device=device)

        # Get to correct shape, 28x28->784
        # -1 will flatten all outer dimensions into one
        data = data.reshape(data.shape[0], -1) 

        # forward propagation
        scores = model(data)
        loss = criterion(scores, targets)

        # zero previous gradients
        optimizer.zero_grad()
        
        # back-propagation
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

Epoch: 0 Train_accuracy: 4.107142857142857 Cross_validation_accuracy: 4.488095238095238
Epoch: 1 Train_accuracy: 83.51984126984127 Cross_validation_accuracy: 83.05952380952381
Epoch: 2 Train_accuracy: 85.78968253968254 Cross_validation_accuracy: 85.19047619047619
Epoch: 3 Train_accuracy: 86.85714285714286 Cross_validation_accuracy: 85.66666666666667
Epoch: 4 Train_accuracy: 87.53968253968254 Cross_validation_accuracy: 86.22619047619048
Epoch: 5 Train_accuracy: 88.0 Cross_validation_accuracy: 86.55952380952381
Epoch: 6 Train_accuracy: 88.52380952380953 Cross_validation_accuracy: 86.96428571428572
Epoch: 7 Train_accuracy: 88.54761904761905 Cross_validation_accuracy: 87.05952380952381
Epoch: 8 Train_accuracy: 88.95238095238095 Cross_validation_accuracy: 87.05952380952381
Epoch: 9 Train_accuracy: 88.94444444444444 Cross_validation_accuracy: 87.22619047619048
Epoch: 10 Train_accuracy: 88.89285714285714 Cross_validation_accuracy: 86.94047619047619
Epoch: 11 Train_accuracy: 96.21428571428572 

In [35]:
check_accuracy(test_loader, model)

96.10714285714286